

```
# This is formatted as code
```


# Supplementary material

Authors:  S. Kokabisaghi and E.J. Pauwels 

Affiliation:  Centrum Wiskunde & Informatica (CWI), Amsterdam, Netherlands.

This notebook provides supplementary material supporting the exposition in the paper: 

S. Kokabisaghi, A. Dorsman, E.J. Pauwels: 
Transitions in sniping behaviour among competing high-frequency traders 

Date:  1 Sept 2020

Version:  2.0




In [1]:
# Required Libraries
import numpy as np
from sympy import *
import pandas as pd # for table

import matplotlib.pyplot as plt # for plotting 
from matplotlib import rcParams

from IPython.display import Latex 
import copy # To clone dictionaries


In [2]:
# Libraries for plotting
plt.rcParams["font.weight"] = "bold"# Bold text in plot
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["figure.figsize"] = [10,8]
import warnings
warnings.filterwarnings("ignore")

**Structure:**

0. [**Symbols and initialised parameters**](#scrollTo=gS6RY01JLRlA&line=23&uniqifier=1): 
    This section introduces primary parameters as both symbols and values.
1. [**Utility Table**](#scrollTo=o2spT3G58KNS&line=4&uniqifier=1) represents possible event codes and corresponding payoffs and utility including:

  1.1. [General case](#scrollTo=o2spT3G58KNS&line=18&uniqifier=1)

  1.2. [Simplified version](#scrollTo=2D6f5S2bIEGs&line=2&uniqifier=1)
  
2. [**Compute the Expected utility for Bandit and market maker**](#scrollTo=Jpm4KxHz9byQ&line=7&uniqifier=1): 

    Compute expected utility and endpoints (A, B) for bandits and (C,D) for market maker using events table.

* 2.0. [**Preamble: probabilities $h(p)$ and $g(p)$**]((#scrollTo=Jpm4KxHz9byQ&line=7&uniqifier=1)) Introduce sniping probability for bandit and market maker.
* 2.1. [**Expected utility for bandit**](#scrollTo=P6L-Tibyb8v9&line=11&uniqifier=1) Compute expected utility and utility endpoints (C, D) for bandit.

* 2.2. [**Expected utility for Market maker**](#scrollTo=nMxvXE2kKcuL)
Compute expected utility and utility endpoints (C, D) for Market maker.


3. [**Dependency of Utility to probability :**](#scrollTo=j5BvMTMyqSGw&line=1&uniqifier=1) Compute the derivatives of probability and endpoints.

4. [**Equilibrium/ Indifference point :**](#scrollTo=KzkqNtByL9bg&line=4&uniqifier=1) Computes equilibrium spread and utility

5. [**Condition on K($\gamma$)**](#scrollTo=2WpKIm1kkGsa&line=3&uniqifier=1) Compute the transition thresholds from sure to probabilistic sniping (threshold $\overline\gamma_K$ )

  5.0. [cubic polynomial:]( #scrollTo=MlLx6PvINVwP&line=2&uniqifier=1
) Expand the cubic polynomial in risk aversion factor $\gamma$
  
  5.1. [Properties of the cubic polynomial in $\gamma$](#scrollTo=-ubEdlnuIRoY&line=4&uniqifier=1) Properties of the cubic polynomial in $\gamma$
     
     5.1.0. [$K_3<0$](#scrollTo=qk07uIjs9l5I&line=26&uniqifier=1) 

     5.1.1.[K($\gamma$)>0](#scrollTo=kwyX7qi0c0-L&line=4&uniqifier=1): $K_3+K_2+K_1+K_0 > 0$
  
  5.1.2.[Cubic polynomial- zero crossing ($\gamma$>1)](#scrollTo=lxN6dgPWWUzd&line=3&uniqifier=1) Prove the solution (polynomial roots >1)

6. [**Condition on L($\gamma$)**](#scrollTo=ExDQxh6cyCKS&line=6&uniqifier=1): Compute and prove the transition from probabilistic to no sniping (threshold $\overline\gamma_L$)











 

**Notations and endpoints for each agent's utility:** (KPD: section 2.2)


1.   $EU_B(s=0) = A \quad$   and  $\quad EU_B(s=\sigma) = B $
2.   $EU_M(s=0) = C \quad $  and $\quad EU_M(s=\sigma) = D $

 $$ EU_B(s)=(1-\frac{s}{\sigma}) A + (\frac{s}{\sigma}) B \\
EU_M(s) = (1-\frac{s}{\sigma}) C+ (\frac{s}{\sigma}) D$$












# 0. Introduce symbols and values

In this notebook we will perform both symbolic and numerical calculations, for which we here introduce the symbols and some values. 

In [3]:
# Introduce the primary parameters as Symbol

from sympy.abc import s, alpha, mu, delta, gamma, sigma, p, H  

# introduce the symbols for secondary parameters (are functions of the primary parameters)

from sympy.abc import beta, g,h,q, m 
alphabar = symbols('alphabar')
mubar = symbols('mubar')
thetabar = symbols('thetabar')
m = 1-mubar




# 1. Define the utility tables

## 1.1  General case

Encoding the table 1 in MZ paper, which we will use to compute the expected utility for both the market maker ($U_M$) and bandit ($U_B$)

In [4]:


#  Encode the pay-off table (MZ paper p. 1201) as a list of dictionaries: 
#-------------------------------------------------------------------------- 
#  REMARKS:  
#  -  The events in the table below are mutually exclusive.  This makes it possible to 
#     decide which events yield a positive or negative outcome. In the latter case we 
#     need to multiply by gamma (risk aversion factor) 
#  -  Notice that we represent the fraction 1/2 as Rational(1,2) in order to streamline computations with fractions! 
#  -  The event codes are self-explanatory: e.g.  
#         NG-LA = trigger =  good news, 2nd event = Liquidity trader on Ask etc.
#         NG/NB : good or bad news,  LA/LB: liquidity trader on ask/bid,  NO = no 2nd event
#  The subscript _rwy means that the payoff is valid if the agent is the racewinner. 
#        Similarly _rwn means that the agent lost the race.    
# 
#  This table is essenially taken from the MZ paper




mz_table_full = [{'eventcode':'NG-LA' , 'prob_trigger':Rational(1,2)*beta, 'prob_2nd_event':mubar, 'util_mm_rwy':gamma*(s-sigma), 'util_mm_rwn':gamma*(s-sigma)  , 'util_bandit_rwy': 0 , 'util_bandit_rwn': 0},
            {'eventcode':'NG-LB' , 'prob_trigger':Rational(1,2)*beta, 'prob_2nd_event': mubar, 'util_mm_rwy':(s+sigma), 'util_mm_rwn':2*s,'util_bandit_rwy' : (sigma-s),'util_bandit_rwn': 0},
            {'eventcode':'NG-NG' , 'prob_trigger':Rational(1,2)*beta, 'prob_2nd_event':alphabar, 'util_mm_rwy': 0 , 'util_mm_rwn': gamma*(s -2*sigma) ,'util_bandit_rwy': 2*sigma-s,'util_bandit_rwn': 0} ,           
            {'eventcode':'NG-NB' , 'prob_trigger':Rational(1,2)*beta, 'prob_2nd_event':alphabar, 'util_mm_rwy': 0,'util_mm_rwn': s, 'util_bandit_rwy': -s * gamma, 'util_bandit_rwn': 0},            
            {'eventcode':'NG-NO' , 'prob_trigger':Rational(1,2)*beta, 'prob_2nd_event':(1-2*(alphabar+mubar)), 'util_mm_rwy':0, 'util_mm_rwn': gamma*(s -sigma), 'util_bandit_rwy':sigma-s, 'util_bandit_rwn':0},
            {'eventcode':'NB-LA' , 'prob_trigger':Rational(1,2)*beta, 'prob_2nd_event':mubar, 'util_mm_rwy': s+sigma,'util_mm_rwn':2*s, 'util_bandit_rwy':sigma -s,'util_bandit_rwn': 0 },
            {'eventcode':'NB-LB' , 'prob_trigger':Rational(1,2)*beta, 'prob_2nd_event':mubar, 'util_mm_rwy': gamma*(s-sigma), 'util_mm_rwn':gamma*(s-sigma), 'util_bandit_rwy':0, 'util_bandit_rwn': 0},    
            {'eventcode':'NB-NG' , 'prob_trigger':Rational(1,2)*beta, 'prob_2nd_event':alphabar, 'util_mm_rwy':0 ,'util_mm_rwn': s, 'util_bandit_rwy': -s *gamma, 'util_bandit_rwn': 0 },
            {'eventcode':'NB-NB' , 'prob_trigger':Rational(1,2)*beta, 'prob_2nd_event':alphabar, 'util_mm_rwy':0 , 'util_mm_rwn': gamma*(s-2*sigma) , 'util_bandit_rwy':(2*sigma-s) , 'util_bandit_rwn':0},
            {'eventcode':'NB-NO' , 'prob_trigger':Rational(1,2)*beta, 'prob_2nd_event':(1-2*(alphabar+mubar)), 'util_mm_rwy': 0, 'util_mm_rwn':  gamma*(s-sigma), 'util_bandit_rwy':(sigma-s) , 'util_bandit_rwn': 0},
            {'eventcode':'LA-LA' , 'prob_trigger':Rational(1,2)*(1-beta),'prob_2nd_event':mubar, 'util_mm_rwy':s , 'util_mm_rwn': s, 'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LA-LB' , 'prob_trigger':Rational(1,2)*(1-beta),'prob_2nd_event':mubar, 'util_mm_rwy': 2*s, 'util_mm_rwn': 2*s,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LA-NG' , 'prob_trigger':Rational(1,2)*(1-beta),'prob_2nd_event':alphabar, 'util_mm_rwy': gamma*(s-sigma), 'util_mm_rwn': gamma*(s-sigma),  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LA-NB' , 'prob_trigger':Rational(1,2)*(1-beta),'prob_2nd_event':alphabar, 'util_mm_rwy': s +sigma, 'util_mm_rwn': s+ sigma,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LA-NO' , 'prob_trigger':Rational(1,2)*(1-beta),'prob_2nd_event':(1-2*(alphabar+mubar)), 'util_mm_rwy': s, 'util_mm_rwn': s,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LB-LA' , 'prob_trigger':Rational(1,2)*(1-beta),'prob_2nd_event':mubar, 'util_mm_rwy': 2*s, 'util_mm_rwn': 2*s,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LB-LB' , 'prob_trigger':Rational(1,2)*(1-beta),'prob_2nd_event':mubar, 'util_mm_rwy': s, 'util_mm_rwn': s,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LB-NG' , 'prob_trigger':Rational(1,2)*(1-beta),'prob_2nd_event':alphabar, 'util_mm_rwy': s+ sigma, 'util_mm_rwn':  s+ sigma,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LB-NB' , 'prob_trigger':Rational(1,2)*(1-beta),'prob_2nd_event':alphabar, 'util_mm_rwy': gamma*(s -sigma), 'util_mm_rwn':  gamma*(s -sigma),  'util_bandit_rwy':0 , 'util_bandit_rwn': 0},
            {'eventcode':'LB-NO' , 'prob_trigger':Rational(1,2)*(1-beta),'prob_2nd_event':(1-2*(alphabar+mubar)), 'util_mm_rwy':s, 'util_mm_rwn':  s,  'util_bandit_rwy':0 , 'util_bandit_rwn': 0}]



Table =pd.DataFrame(mz_table_full)
pd.set_option('display.max_columns', 8)
pd.set_option('display.width', 180)
print(Table)

    



   eventcode   prob_trigger             prob_2nd_event        util_mm_rwy          util_mm_rwn util_bandit_rwy  util_bandit_rwn
0      NG-LA         beta/2                      mubar  gamma*(s - sigma)    gamma*(s - sigma)               0                0
1      NG-LB         beta/2                      mubar          s + sigma                  2*s      -s + sigma                0
2      NG-NG         beta/2                   alphabar                  0  gamma*(s - 2*sigma)    -s + 2*sigma                0
3      NG-NB         beta/2                   alphabar                  0                    s        -gamma*s                0
4      NG-NO         beta/2  -2*alphabar - 2*mubar + 1                  0    gamma*(s - sigma)      -s + sigma                0
5      NB-LA         beta/2                      mubar          s + sigma                  2*s      -s + sigma                0
6      NB-LB         beta/2                      mubar  gamma*(s - sigma)    gamma*(s - sigma)          

## 1.2  Simplify table by setting sigma = 1

Since the jumpsize $\sigma$ (sigma) is in fact a scaling parameter we can, without loss of generality, fix it to one:  (sigma = 1). 
For the rest of this notebook, we will use this convention. 




In [5]:
# Make a copy of the table and substitute sigma = 1

mz_table = mz_table_full.copy()

for d in mz_table: 
  for x,y in d.items():
    #  substitution only works for expressions, exclude the rest
    if type(y) != str  and type(y) != int:
      y_new = y.subs({sigma:1})
      d.update({x:y_new})


Table = pd.DataFrame(mz_table)
pd.set_option('display.max_columns', 8)
pd.set_option('display.width', 180)
print("=======================  Utility table (simplified for sigma = 1) =======================")
print(' ')
print(Table)



=======================  Utility table (simplified for sigma = 1) =======================
 
   eventcode   prob_trigger             prob_2nd_event    util_mm_rwy    util_mm_rwn util_bandit_rwy  util_bandit_rwn
0      NG-LA         beta/2                      mubar  gamma*(s - 1)  gamma*(s - 1)               0                0
1      NG-LB         beta/2                      mubar          s + 1            2*s          -s + 1                0
2      NG-NG         beta/2                   alphabar              0  gamma*(s - 2)          -s + 2                0
3      NG-NB         beta/2                   alphabar              0              s        -gamma*s                0
4      NG-NO         beta/2  -2*alphabar - 2*mubar + 1              0  gamma*(s - 1)          -s + 1                0
5      NB-LA         beta/2                      mubar          s + 1            2*s          -s + 1                0
6      NB-LB         beta/2                      mubar  gamma*(s - 1)  gamma*(s - 

# 2. Compute expected utility $EU_B $ for **bandit** and  $EU_M$ for **market maker**


To compute the expected utility, we need to probability of sniping.



## 2.0  Preamble: probabilities $h(p)$ and $g(p)$
**For more information: KPD.section 3.2, Appendix E**

The expected utilities for bandit and market maker depend on three probabilities which we will introduce here.  For the detailed 
computation of these quantities we refer to KPD_supplementary_material_2.ipynb

When the first event (i.e. trigger event) in the MZ game is the arrival 
of a news event, the market maker (MM) will race to update his stale quotes. 
The bandits however will only enter the race with probability $p$. 
This probability $p$ determines the values of two dependent probabilities: 
 

  1. $h(p) = P(\mbox{MM will lose the race})$
  2. $g(p) = P(\mbox{bandit will win race} \,| \, \mbox{he enters the race})$

In the paper (see section 3.2 and appendix E) it is shown that 

$$ h(p) = \frac{(1-p)^H-(1-Hp)}{Hp}  \quad \quad \mbox{and} \quad\quad 
g(p) = \frac{h(p)}{(H-1)p}  $$






## 2.1. To compute expected utility for bandits $EU_B$: 
**For more information: KPD.section 3.3.1, Appendix D**


we consider the eventcodes that he wins the race. Therefore, we have to condition three events: 




1.  There is a race: prob = $\beta$
2.  The bandit enters the race: prob = $p$
3.  He wins the race (given that he 
    entered): prob = $g(p)$  

Using this information we can make the 
following factorisation: 

$$\begin{eqnarray}
  EU_B(s) &=&\beta p g(p) EU_B(s | \mbox{bandit enters and wins race}) \\
  & = & \frac{1}{2}\beta p g(p) \sum_{e_1 \in N*} \sum_{e_2 } 
    p(e_2) \,  u_B(s |  e_2 \,\&\, \mbox{bandit wins race})\nonumber\\
    &=& \beta p g(p) S
\end{eqnarray}$$

Where:
$$ S := \frac{1}{2}\sum_{e_1 \in N*} \sum_{e_2} p(e_2)\,u_B(s| e \,\&\,\mbox{bandit wins}) = 
\overline{\alpha}\,s (1-\gamma) + (\overline{\mu}(s-1)-s+1)
$$


Compute the endpoints of the line segments:

Next, compute the values for EU_B for s=0 and s=$\sigma$=1 

$$ 
  \begin{array}{lclclcl}
 A &:=& EU_B(s=0) &=& \beta p g(p) (1-\overline{\mu}) &=&  (m \,\beta)\, p  g(p)\\
 B &:=& EU_B(s=1)  &=& -\overline{\alpha}(\gamma-1)  \beta p g(p) 
 &=& (-\overline{\alpha}q \,\beta) \, p g(p)
\end{array}
$$
 


In [19]:
# beta = alpha/(alpha+mu)
# alphabar = Rational(1,2)*alpha*delta
# mubar = Rational(1,2)*mu*delta
# thetabar = alphabar*(1-beta) + mubar*beta # 2*alphabar*mubar/(alphabar + mubar)

EU_B = symbols('EU_B')
g = Symbol('g')

EU_B = 0
for d in mz_table: 
  # EU_B += d['util_bandit_rwy']*d['prob_2nd_event']
  EU_B += d['util_bandit_rwy']*d['prob_trigger']*d['prob_2nd_event']

print(' ==== Computation of A := EU_B(s=0) and B := EU_B(s=1) ============= \n')

A = EU_B.subs({s:0})

B = EU_B.subs({s:1})

#  Print after some algebraic post-processing: 
A = collect(A.expand(), beta)
B = B.expand().collect(beta).collect(alphabar)
print(' After simplification, we obtain the formula in KPD eq. 25 :\n ')

print('A = EU_B(s=0) = ', A)
print('B = EU_B(s=1) = ', B)

print('')
EU_B = p*g*EU_B
EU_B = simplify(EU_B)
print('EU_B = ', EU_B)

 ==== Computation of A := EU_B(s=0) and B := EU_B(s=1) ============= 

 After simplification, we obtain the formula in KPD eq. 25 :
 
A = EU_B(s=0) =  beta*(-mubar + 1)
B = EU_B(s=1) =  alphabar*beta*(-gamma + 1)

EU_B =  beta*g*p*(-alphabar*gamma*s + alphabar*s + mubar*s - mubar - s + 1)


## 2.2. Compute expected utility $EU_M$ for **market maker** 
**(For more details: See: KPD.section 3.3.2, Appendix D)**

To compute the expected value $EU_M$: 

- we first condition on the nature of the 
trigger event: news (prob = $\beta$) or 
LT arrival (prob = $1-\beta$). 

- Next, in case the trigger event is news,
 a race ensues, and we additionally condition on whether (prob: $1-h$) or not (prob. $h$) the market maker wins.

  Explicitly splitting an eventcode $e$ 
into the first and 
second event $e = (e_1,e_2)$ (e.g. for 
$e= $ {LA-NG } we have, $e_1 = $ { LA } and $e_2= $ { NG}. 
Referring to the probability tree in 
KPD.Fig.15 we see that the probability of 
each eventcode $e$ can be decomposed as: 

$$  
p(e) = p(e_1) p(e_2) = 
\left\{\begin{array}{lcl}
{\displaystyle \left(\beta\cdot \frac{1}{2}\right) \cdot p(e_2)} & \quad & \mbox{if }  e \in \mbox{{ N***} }\\[2ex]
{\displaystyle \left((1-\beta)\cdot \frac{1}{2}\right) \cdot p(e_2)} & \quad & \mbox{if }  e \in \mbox{{  L***} }
\end{array}
\right.
$$
Hence, we get the following 
expansion in terms of possible event-codes $e$: 
      $$\begin{eqnarray*}
      EU_M(s) &= & \beta\underbrace{\left[ \sum_{e \in N***} p(e) \,u_M(s | e 
      ) \right]}_{\mbox{Trigger = News (race!)}} +  (1-\beta)\underbrace{\sum_{e \in L***} p(e) \,u_M(s| e)]}_{\mbox{Trigger = LT  (no race!)}}\\
      &=& \beta [\frac{1}{2} \sum_{e_1\in N*}\sum_{e_2 } p(e_2) \,u_M(s | e ) ] +  \quad (1-\beta)  [  \frac{1}{2} \sum_{e_1\in L*}\sum_{e_2 } p(e_2) \,u_M(s | e ) ]\\&=& \frac{\beta}{2}[\sum_{e_1 \in N*}\sum_{e_2}p(e_2) \cdot h \cdot u_M(s |e \, \& \,\mbox{MM loses})+ (1-h) \cdot  u_M(s | e \,\&\,\mbox{MM wins})]\\  && \quad\quad\quad + \, 
          \frac{(1-\beta)}{2} \sum_{e_1\in L*}\sum_{e_2 } \left\{p(e_2) \,u_M(s
          | e)  \right\}  \nonumber
      \end{eqnarray*}$$
      
To Compute the end points: 

$$  C := EU_M(s = 0) \quad \quad \mbox{and} \quad \quad 
D := EU_M(s=1)   $$









In [7]:

h = Symbol('h')
EU_M = symbols('EU_M')

#  Use the same methodology for EU_M (expected utility of market maker)

EU_M_rwy = 0
EU_M_rwn = 0

for d in mz_table: 
  EU_M_rwy += d['util_mm_rwy']*d['prob_trigger']*d['prob_2nd_event']
  EU_M_rwn += d['util_mm_rwn']*d['prob_trigger']*d['prob_2nd_event']

#  Specify for the endpoints

Cy = EU_M_rwy.subs({s:0}).expand()
Cn = EU_M_rwn.subs({s:0}).expand()

Dy = EU_M_rwy.subs({s:1}).expand()
Dn = EU_M_rwn.subs({s:1}).expand()

# The actual utility needs to be multiplied with prob that 
# MM will win (1-h) or lose (h) the race: 
# C = (1-h)*Cy + h*Cn
# D = (1-h)*Dy + h*Dn
#
#  which is re-arranged as: 

C = Cy + (Cn-Cy)*h
D = Dy + (Dn-Dy)*h

#####################
#  Expanding C
#####################

print('\n ==== Computation of C := EU_M(s=0) ========== \n')

#  These terms can be further simplified: 
Cd = Cn-Cy 
Cd = Cd.subs(gamma,q+1)
Cd = Cd.cancel().collect(beta)

print('Cd = Cn-Cy = ', Cd)
print('-----------------')
beta_eq = alpha/(alpha+mu)
Cy = Cy.subs({beta:beta_eq, gamma:q+1})

#  Invoke the definitions of alphabar, mubar and thetabar to simplify the expansion

Cy = Cy.subs({alpha: 2*alphabar/delta,mu: 2*mubar/delta})
Cy = Cy.cancel().collect(q)
Cy = Cy.simplify()
# Use the definition of thetabar to substitute
Cy = Cy.subs({2*alphabar*mubar/(alphabar+mubar) : thetabar})

#Cy = Cy.subs({gamma:q+1}).expand().collect(q).collect(alphabar).collect(-1)
# Use the definition of thetabar to substitute
print('Cy =', Cy)    

print('-----------------')

C = Cy + Cd*h

print('C = EU_M(s=0) = Cy + Cd*h = ', C)

print('\n   We next show that 2*mubar*q+mubar-q-1 = mubar*q - m*gamma = mubar*q-m*(q+1)) ')
print('   by proving that the difference is zero: ')
T = cancel(2*mubar*q+mubar-q-1 - (mubar*q-m*(q+1)) )
print('\n   The difference between 2*mubar*q+mubar-q-1 and (mubar*q-m*(q+1)) = ', T.expand())
print('\n   Hence we obtain the formula in KDP eq. 30 : ')
print('\n       C = EU_M(s=0) = beta*h*(mubar*q  - m*gamma) - q*thetabar  ')

################################
# Expanding D
################################

print('\n\n ==== Computation of D := EU_M(s=1) ============= \n')

D = D.subs(gamma,q+1).cancel().collect(beta)
print('D = EU_M(s=1) = ', D) #D.subs(gamma,q+1).cancel().collect(beta))

print('\n Recall that m = 1-mubar')
print(' Hence we obtain the formula in KPD eq. 30 : ')
print('\n       D = EU_M(s=1) =  -beta*(alphabar*h*q + m) + (mubar + 1)')
print('')
EU_M = (1-h)*EU_M_rwy + h* EU_M_rwn
print('EU_M = ', simplify(EU_M))



 ==== Computation of C := EU_M(s=0) ========== 

Cd = Cn-Cy =  beta*(2*mubar*q + mubar - q - 1)
-----------------
Cy = -q*thetabar
-----------------
C = EU_M(s=0) = Cy + Cd*h =  beta*h*(2*mubar*q + mubar - q - 1) - q*thetabar

   We next show that 2*mubar*q+mubar-q-1 = mubar*q - m*gamma = mubar*q-m*(q+1)) 
   by proving that the difference is zero: 

   The difference between 2*mubar*q+mubar-q-1 and (mubar*q-m*(q+1)) =  0

   Hence we obtain the formula in KDP eq. 30 : 

       C = EU_M(s=0) = beta*h*(mubar*q  - m*gamma) - q*thetabar  


 ==== Computation of D := EU_M(s=1) ============= 

D = EU_M(s=1) =  beta*(-alphabar*h*q + mubar - 1) + mubar + 1

 Recall that m = 1-mubar
 Hence we obtain the formula in KPD eq. 30 : 

       D = EU_M(s=1) =  -beta*(alphabar*h*q + m) + (mubar + 1)

EU_M =  -alphabar*beta*gamma*h*s - alphabar*beta*gamma*s + alphabar*beta*gamma + alphabar*beta*h*s + alphabar*beta*s - alphabar*beta + alphabar*gamma*s - alphabar*gamma - alphabar*s + alphabar - 2*beta*ga

# 3. Dependence of utility (endpoints) on sniping probability $p$ 

**(KPD.section. 3.3.4)**

The endpoints A, B, C and D depend on p explicitly (at least in the case of A and B) as well as implicitly through the functions g(p) and h(p): 

$$ A(p) = m\beta p g(p) \Longrightarrow \frac{dA}{dp} = m\beta\,(g(p) + p g'(p))  $$


$$ B(p) = -\overline{\alpha} q\beta p g(p) \Longrightarrow \frac{dB}{dp} = -\overline{\alpha} q \beta \,(g(p) + p g'(p))  $$

$$ C(p) = - \left\{ q \overline{\theta} + \beta(m\gamma -\overline{\mu}q)h(p)\right\} \Longrightarrow \frac{dC}{dp} = 
\frac{dC}{dh} \frac{dh}{dp}  = -\beta(m\gamma - \overline{\mu}q) \, h'(p)$$

$$ D(p) = (1+\overline{\mu}) - \beta (m+\overline{\alpha} q h(p)) 
\Longrightarrow \frac{dD}{dp} = 
\frac{dD}{dh} \frac{dh}{dp} = - \overline{\alpha} \beta q \, h'(p)  $$

Hence, to compute the required conditions, we need the values 
of these derivatives only 
for p=0 and p=1. 

In [8]:
# Define the endpoints A,B,C and D as functions of individual sniping probability p
#  as well as the corresponding derivatives 
#
#  IN FACT WE DO THE EXPLICIT COMPUTATIONS IN THE APPENDIX 

#  The detailed computation of the functions h(p) and g(p) is discussed in 
#  Appendix E of the paper. 
#  Here we just copy the results for g, h and its derivatives. 

h = ( p*H - 1 + (1-p)**H)/ (p*H)
g = h/((H-1)*p)
dg_dp = symbols('dg_dp')
dh_dp = symbols('dh_dp')

h_0 = 0
h_1 = (H-1)/H
dh_dp_0 = Rational(1,2)*(H-1)    #(H-1)/2
dh_dp_1 = 1/H

g_0 = Rational(1,2)  # 1/2
g_1 = 1/H
dg_dp_0 = Rational(-1,6)*(H-2)  # - (H-2)/6
dg_dp_1 = -(H-2)/(H*(H-1))

#  Bandit:  A -endpoint
#------------ 
print('====== BANDIT: =========')
print('')

A_p = m*beta*p*g      #  notice that A_1 = A (intercept for sure sniping)
dA_dp = m*beta*(g+p*dg_dp)

#  ISSUE: SUBSTITUTING p = 0 yields nan because formally g_p has p in denominator
#  for that reason we hardcode A_0
A_0 = 0  # A_p.subs({p:0, g:g_0}) 
A_1 = (1-mubar)*beta/H     # A_p.subs({p:1, g:g_1})

#print(A_p)
print('A(p=0) = ', A_0)
print('A(p=1) = ', A_1)
#print('Sanity check: A(p=1) should be equal to A = ', A)


dA_dp_0 = dA_dp.subs({p:0, g:g_0, dg_dp: dg_dp_0})
dA_dp_1 = dA_dp.subs({p:1, g:g_1, dg_dp: dg_dp_1})

print("A'(0) = ", dA_dp_0)
print("A'(1) = ", dA_dp_1)



#  Bandit:  B -endpoint
#-------------------------------------
B_p = -alphabar*q*beta*p*g
dB_dp = -alphabar*q*beta*(g + p* dg_dp)

B_0 = 0
B_1 = -alphabar*q*beta*g_1 # B_p.subs({p:1, g:g_1})

dB_dp_0 = dB_dp.subs({p:0, g:g_0, dg_dp: dg_dp_0})
dB_dp_1 = dB_dp.subs({p:1, g:g_1, dg_dp: dg_dp_1})

print('')
print('B(0) = ', B_0)
print('B(1) = ', B_1)
print("B'(0) = ", dB_dp_0)
print("B'(1) = ", dB_dp_1)

#------------------------------
# Market maker
#-------------------------------

print('')
print('======= MARKET MAKER ========')
print('') 

C_p = -(q*thetabar + h*beta*(m*gamma-mubar*q))
dC_dp = -beta*(m*gamma -mubar*q)* dh_dp

C_0 = C_p.subs({h:h_0})
C_1 = C_p.subs({h:h_1})
dC_dp_0  = dC_dp.subs({dh_dp:dh_dp_0})
dC_dp_0 = dC_dp_0.factor()
dC_dp_1  = dC_dp.subs({dh_dp:dh_dp_1})

print("C(0) = ", C_0)
print("C(1) = ", C_1)

print("C'(0) = ", dC_dp_0)
print("C'(1) = ", dC_dp_1)

# Endpoint D

D_p = (1+mubar) - beta*(m + alphabar*q*h)
dD_dp = - beta*alphabar*q*dh_dp

D_0 = D_p.subs({h:h_0})
D_1 = D_p.subs({h:h_1})
dD_dp_0 = - beta*alphabar*q*dh_dp_0
dD_dp_0 = dD_dp_0.factor()
dD_dp_1 = - beta*alphabar*q*dh_dp_1

print('')
print("D(0) = ", D_0)
print("D(1) = ", D_1)
print("D'(0) = ", dD_dp_0)
print("D'(1) = ", dD_dp_1)




====== BANDIT: =========

A(p=0) =  0
A(p=1) =  beta*(-mubar + 1)/H
A'(0) =  beta*(-mubar + 1)/2
A'(1) =  beta*(-mubar + 1)*((-H + 2)/(H*(H - 1)) + 1/H)

B(0) =  0
B(1) =  -alphabar*beta*q/H
B'(0) =  -alphabar*beta*q/2
B'(1) =  -alphabar*beta*q*((-H + 2)/(H*(H - 1)) + 1/H)

======= MARKET MAKER ========

C(0) =  -q*thetabar
C(1) =  -q*thetabar - beta*(H - 1)*(gamma*(-mubar + 1) - mubar*q)/H
C'(0) =  beta*(H - 1)*(gamma*mubar - gamma + mubar*q)/2
C'(1) =  -beta*(gamma*(-mubar + 1) - mubar*q)/H

D(0) =  -beta*(-mubar + 1) + mubar + 1
D(1) =  -beta*(-mubar + 1 + alphabar*q*(H - 1)/H) + mubar + 1
D'(0) =  -alphabar*beta*q*(H - 1)/2
D'(1) =  -alphabar*beta*q/H


# 4.  Spread $s^*$ and $u^*$ utility at the point of indifference:
**(KPD. section 3.3.3)**

**Equilibrium/indifference spread and utility :** 


(KPD.eq.31)

$$ s^*(p) =  \frac{A(p) - C(p)}{(A(p)-C(p))+(D(p)-B (p))}$$


(KPD.eq.32)
$$ \begin{eqnarray}
   u^*(p)  & = & \frac{A(p)\,D(p)-B(p)\,C(p)}{(A(p)-C(p))+(D(p)-B(p))}
\end{eqnarray}$$ 

In [9]:
# Computing optimal utility (U*(p)) at the point of indifference
N =  A - C + D - B
Q = A*D - B*C

# Optimal utility is the following:
u_star =( A *D - B*C)/N
print('Indifference Utility u*(p):')
print(u_star)

# Optimal spread:

s_star = (A - C)/ N
print('\n Indifference Spread s*(p) = ')
print(s_star)


Indifference Utility u*(p):
(-alphabar*beta*(-gamma + 1)*(beta*h*(2*mubar*q + mubar - q - 1) - q*thetabar) + beta*(-mubar + 1)*(beta*(-alphabar*h*q + mubar - 1) + mubar + 1))/(-alphabar*beta*(-gamma + 1) - beta*h*(2*mubar*q + mubar - q - 1) + beta*(-mubar + 1) + beta*(-alphabar*h*q + mubar - 1) + mubar + q*thetabar + 1)

 Indifference Spread s*(p) = 
(-beta*h*(2*mubar*q + mubar - q - 1) + beta*(-mubar + 1) + q*thetabar)/(-alphabar*beta*(-gamma + 1) - beta*h*(2*mubar*q + mubar - q - 1) + beta*(-mubar + 1) + beta*(-alphabar*h*q + mubar - 1) + mubar + q*thetabar + 1)


# 5. Transition from sure to probabilistic sniping ($\overline\gamma_K$ ) 

**(KPD.section.4.1, Appendix F)**

The transition from pure to probabilistic sniping occurs when 
 $$ \left. \frac{du^*}{dp} \right|_{(p=1)} = 0 $$

Introducing short-hand notation for the numerator and denominator for $u^*(p)$  (see above section(4)) we get: 

 $$u^*(p)   =  \frac{A(p)\,D(p)-B(p)\,C(p)}{(A(p)-C(p))+(D(p)-B(p))} \equiv \frac{N(p)}{Q(p)}, $$

whence


$$  \frac{du^*}{dp} = \frac{N'(p)Q(p) - N(p) Q'(p)}{Q^2(p)} $$

From this we see that the transition condition can be recast as: 

(KPD.eq.40)
$$  N'(1)Q(1) - N(1)Q'(1) = 0.   $$

We first compute all the relevant factors in the LHS of this condition. 


In [10]:
#  Incrementally building the conditions for the thresholds


N_0 = A_0*D_0 - B_0*C_0
N_1 = A_1*D_1 - B_1*C_1
Q_0 = A_0 - C_0 + D_0 - B_0
Q_1 = A_1 - C_1 + D_1 - B_1


dN_dp = ( (dA_dp*D_p)+ (A_p*dD_dp)) - ((dB_dp*C_p)+(B_p*dC_dp))
dN_dp_0 =  ( (dA_dp_0*D_0)+ (A_0*dD_dp_0)) - ((dB_dp_0*C_0)+(B_0*dC_dp_0))
dN_dp_1 =  ( (dA_dp_1*D_1)+ (A_1*dD_dp_1)) - ((dB_dp_1*C_1)+(B_1*dC_dp_1))

dQ_dp = dA_dp - dC_dp + dD_dp - dB_dp
dQ_dp_0 = dA_dp_0 - dC_dp_0 + dD_dp_0 - dB_dp_0
dQ_dp_1 = dA_dp_1 - dC_dp_1 + dD_dp_1 - dB_dp_1

print('')
print('  To compute du/dp_1 we need N_1,Q_1, dN_dp_1 and dQ_dp_1 ')


print('-------------- N_1 ----------------')
N_1 = N_1.expand().cancel()
print('N_1 = ', N_1) 
print('')
#pprint(N_1)

#  Extract numerator and denominator separately

N_1_numer, N_1_denom = N_1.as_numer_denom()
print('Extracting numerator and denominator')
print('')
print('N_1 numerator: ')
print('-- number of factors = ', len(N_1_numer.factor().args)) 
print('-- expansion: ', N_1_numer.factor())
print('N_1 denominator = ') 
print('-- number of factors = ', len(N_1_denom.factor().args)) 
print('-- expansion: ', N_1_denom.factor())
print('------------------------------')


print('')
print('-------------- Q_1 ----------------')
Q_1 = Q_1.expand().cancel()
print('Q_1 = ', Q_1) 
print('')
#pprint(Q_1)

#  Extract numerator and denominator separately

Q_1_numer, Q_1_denom = Q_1.as_numer_denom()
print('Extracting numerator and denominator')
print('')
print('Q_1 numerator: ')
print('-- number of factors = ', len(Q_1_numer.factor().args)) 
print('-- expansion: ', Q_1_numer.factor())
print('Q_1 denominator = ') 
print('-- number of factors = ', len(Q_1_denom.factor().args)) 
print('-- expansion: ', Q_1_denom.factor())
print('------------------------------')



print('')
print('-------------- dN_dp_1 ----------------')
dN_dp_1 = dN_dp_1.expand().cancel()
print('dN_dp_1 = ', dN_dp_1) 
print('')
#pprint(dN_dp_1)

#  Extract numerator and denominator separately

dN_dp_1_numer, dN_dp_1_denom = dN_dp_1.as_numer_denom()
print('Extracting numerator and denominator')
print('')
print('dN_dp_1 numerator: ')
print('-- number of factors = ', len(dN_dp_1_numer.factor().args)) 
print('-- expansion: ', dN_dp_1_numer.factor())
print('dN_dp_1 denominator = ') 
print('-- number of factors = ', len(dN_dp_1_denom.factor().args)) 
print('-- expansion: ', dN_dp_1_denom.factor())
print('------------------------------')

print('')
print('-------------- dQ_dp_1 ----------------')
dQ_dp_1 = dQ_dp_1.expand().cancel()
print('dQ_dp_1 = ', dQ_dp_1) 
print('')
#pprint(dQ_dp_1)

#  Extract numerator and denominator separately

dQ_dp_1_numer, dQ_dp_1_denom = dQ_dp_1.as_numer_denom()
print('Extracting numerator and denominator')
print('')
print('dQ_dp_1 numerator: ')
print('-- number of factors = ', len(dQ_dp_1_numer.factor().args)) 
print('-- expansion: ', dQ_dp_1_numer.factor())
print('dQ_dp_1 denominator = ') 
print('-- number of factors = ', len(dQ_dp_1_denom.factor().args)) 
print('-- expansion: ', dQ_dp_1_denom.factor())
print('------------------------------')







  To compute du/dp_1 we need N_1,Q_1, dN_dp_1 and dQ_dp_1 
-------------- N_1 ----------------
N_1 =  (H*alphabar*beta**2*gamma*mubar*q - H*alphabar*beta**2*gamma*q + H*alphabar*beta**2*mubar*q**2 + H*alphabar*beta**2*mubar*q - H*alphabar*beta**2*q - H*alphabar*beta*q**2*thetabar - H*beta**2*mubar**2 + 2*H*beta**2*mubar - H*beta**2 - H*beta*mubar**2 + H*beta - alphabar*beta**2*gamma*mubar*q + alphabar*beta**2*gamma*q - alphabar*beta**2*mubar*q**2 - alphabar*beta**2*mubar*q + alphabar*beta**2*q)/H**2

Extracting numerator and denominator

N_1 numerator: 
-- number of factors =  2
-- expansion:  beta*(H*alphabar*beta*gamma*mubar*q - H*alphabar*beta*gamma*q + H*alphabar*beta*mubar*q**2 + H*alphabar*beta*mubar*q - H*alphabar*beta*q - H*alphabar*q**2*thetabar - H*beta*mubar**2 + 2*H*beta*mubar - H*beta - H*mubar**2 + H - alphabar*beta*gamma*mubar*q + alphabar*beta*gamma*q - alphabar*beta*mubar*q**2 - alphabar*beta*mubar*q + alphabar*beta*q)
N_1 denominator = 
-- number of factors =  2
-- e

#5.0. Expand as a cubic polynomial in risk aversion factor $\gamma$

The transition from pure to probabilistic sniping is governed by:  
$$  N'(1)Q(1) - N(1)Q'(1) = 0.   $$

From the calculations in the previous cell, we find explicit (and simple)  expressions for the denominators of the four quantities involved (we leave the numerators unspecified for now): 
 
  1.  N(1) = N_1 = N_1_numer/H^2
  2.  Q(1) = Q_1 = Q_1_numer/H 
  3.  N'(1) = dN/dp_1 = dN_dp_1_numer/((H-1)*H^2)
  4.  Q'(1) = dQ/dp_1 = dQ_dp_1_numer((H-1)*H)
 
This implies that 

   1.  N'(1)Q(1) --->  denominator = $(H-1)H^3$
   2.  N(1)Q'(1) --->  denominator = $(H-1)H^3$

The common denominator in condition gamma_K therefore is $(H-1)H^3$. From the expansions above we can conclude that both terms in gamma_K have the same denominator $(H-1)H^3$ and it suffices to add the numerators.  

Next we define the condition for the gamma_K threshold by combining the numerators: 

$$ \mbox{gamma_K_cond = dN_dp_1_numer*Q_1_numer -  N_1_numer*dQ_dp_1_numer }  $$

This turns out to be a cubic polynomial in $\gamma$ for which we need 
to identify the zero-crossings: 

$$ K(\gamma) := K_3 \gamma^3 + K_2 \gamma^2 + K_1 \gamma +K_0 = 0  $$

It turns out that we can simplify slightly by factoring out some common factors: 

$$  K_3 = -\overline{\alpha}\beta K_{30} \quad\quad 
 K_2 = \overline{\alpha}\beta K_{20} \quad\quad 
  K_1 = -\beta K_{10} \quad\quad 
 K_0  = \beta K_{00} 
 $$


In [11]:

#==========================
#  Preliminary conclusion
#===========================


# $$  N'(1)Q(1) - N(1)Q'(1) = 0.   $$

#
#  From the calculations in the previous cell, we find explicit (and simple)  expressions 
#  for the denominators of the four quantities involved: 
# 
#  N_1 = N_1_numer/H^2
#  Q_1 = Q_1_numer/H 
#  dN/dp_1 = dN_dp_1_numer/((H-1)*H^2)
#  dQ/dp_1 = dQ_dp_1_numer((H-1)*H)
# 
# This implies that 
#     N'(1)*Q(1) --->  denominator = (H-1)*H^3
#     N(1)*Q'(1) --->  denominator = (H-1)*H^3
#
#  The common denominator therefore is:   H^2(1-H)^2
#  We can therefore rebuild the condition for gamma_K as follows: 
#  
#  gamma_K_cond = N'(1)*Q(1) - N(1)*Q'(1) = 0
# 
#  From the expansions above we can conclude that both terms in gamma_K 
#  have the same denominator (H-1)*H^3 and it suffices to add the appropriate numerators

gamma_K_cond = dN_dp_1_numer*Q_1_numer -  N_1_numer*dQ_dp_1_numer

gamma_K_cond = gamma_K_cond.subs({q:gamma-1})

# Expand as a polynomial in gamma 

poly_gamma_K_cond = poly_from_expr(gamma_K_cond,gamma)[0]

#  poly_gamma_K_cond = K3*gamma^3 + K2*gamma^2 + K1*gamma + K0

# Produce a list of cofficients (list[0] corresponds to highest degree!)
gamma_K_cond_coeffs = poly_gamma_K_cond.all_coeffs()

print('---------------')
for cc in  gamma_K_cond_coeffs:
  print(cc.factor())
print('----------------')  

#  Zoom in on the coefficient of gamma^3  (i.e. gamma_K_cond_coeffs[0])

K3 = gamma_K_cond_coeffs[0].factor()
K2 = gamma_K_cond_coeffs[1].factor()
K1 = gamma_K_cond_coeffs[2].factor()
K0 = gamma_K_cond_coeffs[3].factor()

print('K3  = ' ,K3)
print('K2  = ' ,K2)
print('K1  = ' ,K1)
print('K0  = ' ,K0)


#  From the listing of the K-coefficients we see that we can further factorize: 
#  K3 = -alphabar*beta*K30
#  K2 =  alphabar*beta*K20
#  K1 = -beta*K10
#  K0 =  beta*K00

#  Extract the corresponding factors: 
K30 = -(K3/(alphabar*beta)).cancel()
K20 = (K2/(alphabar*beta)).cancel()
K10  = -(K1/beta).cancel()
K00  = (K0/beta).cancel()

print('')
print('K30 = ', K30)
print('K20 = ', K20)
print('K10 = ', K10)
print('K00 = ', K00)





---------------
-alphabar*beta*(2*H**2*alphabar*beta**2*mubar - H**2*alphabar*beta**2 + 4*H**2*beta**2*mubar**2 - 4*H**2*beta**2*mubar + H**2*beta**2 - 4*H**2*beta*mubar*thetabar + 2*H**2*beta*thetabar + H**2*thetabar**2 - 6*H*alphabar*beta**2*mubar + 3*H*alphabar*beta**2 - 8*H*beta**2*mubar**2 + 8*H*beta**2*mubar - 2*H*beta**2 + 4*H*beta*mubar*thetabar - 2*H*beta*thetabar + 4*alphabar*beta**2*mubar - 2*alphabar*beta**2 + 4*beta**2*mubar**2 - 4*beta**2*mubar + beta**2)
alphabar*beta*(4*H**2*alphabar*beta**2*mubar - H**2*alphabar*beta**2 + 10*H**2*beta**2*mubar**2 - 9*H**2*beta**2*mubar + 2*H**2*beta**2 + 4*H**2*beta*mubar**2 - 9*H**2*beta*mubar*thetabar + 2*H**2*beta*mubar + 3*H**2*beta*thetabar - 2*H**2*beta - H**2*mubar*thetabar + 3*H**2*thetabar**2 - H**2*thetabar - 12*H*alphabar*beta**2*mubar + 3*H*alphabar*beta**2 - 18*H*beta**2*mubar**2 + 15*H*beta**2*mubar - 3*H*beta**2 - 4*H*beta*mubar**2 + 8*H*beta*mubar*thetabar - 2*H*beta*mubar - 2*H*beta*thetabar + 2*H*beta + 8*alphabar*bet

# 5.1. Properties of the cubic polynomial $K(\gamma)$ 

**(KPD.section 4.3.1)**
$$ K(\gamma) = K_3 \gamma^3 + K_2\gamma^2 + K_1\gamma + K_0  $$



##5.1.0.  $K_3 < 0$

In this section we first show that $K_3 < 0$  which implies that 

$$ \lim_{\gamma\rightarrow \infty} K(\gamma) = -\infty  $$

Since $$K_3 = -\overline{\alpha}\beta K_{30}  $$

we have to show that $K_{30}>0$. 

To this end we recast $K_{30}$ as a quadratic polynomial in $\beta$:

$$  K_{30}  = K_{302}\beta^2 +   K_{301}\beta +  K_{300}$$

Since $0 \leq \beta \leq 1$ it suffices to show that $K_{30} > 0$ over that 
$\beta$-range.  This follows from the observations: 

  1. $K_{302} > 0 $ so the $\beta$-quadratic is convex; 
  2. $K_{30}(\beta=0) = K_{300}> 0$
  3. The minimum of the parabola is located at negative $\beta$-values;  this follows from the fact that $K_{302}> 0$  and the minimum is achieved for 
  $\beta$-value equal to $-K_{301}/(2K_{302}) < 0$.

These observations guarantee that the $\beta$-polynomial is positive 
for positive $\beta$-values. 


In [12]:
#  we have to prove that K30 > 0  (to show that K3 < 0)

#  Express K30 as (quadratic) polynomial 

print('K30 as polynomial in beta')
print('')
poly_K30 = poly_from_expr(K30,beta)[0]
print(poly_K30)

print('')
print('Coefficients of K30 in beta')

print('')
K302 = (poly_K30.all_coeffs()[0]).factor()
K301 = (poly_K30.all_coeffs()[1]).factor()
K300 = (poly_K30.all_coeffs()[2]).factor()

print('K30-coef of beta^2 = K302 = ', K302)
print('K30-coef of beta^1 = K301 = ', K301)
print('K30-coef of beta^0 = K300 = ', K300)

#  Since mubar < 1/2  it follows that (2*mubar-1) < 0.
#  Simlarly:  alphabar + mubar < 1/2   (2*alphabar + 2*mubar -1) < 0  

print('')
print('Clearly K300 > 0')
print('Since (2*mubar-1)< 0 , it follows that K301 > 0')
print(' ')
print('Also: if we denote  R =  (H*alphabar + 2*H*mubar - H - 2*alphabar - 2*mubar + 1)')
print('then by adding H*alphabar, we get')

R = H*alphabar + 2*H*mubar - H - 2*alphabar - 2*mubar + 1
RR = R+alphabar*H 
print(RR.factor())
print('R < R+ H*alphabar = (2H-1) = ', RR.factor()) 

print('Since 2*alphabar+2*mubar-1 < 0, it follows that the RHS < 0  and hence K302 > 0 ')



K30 as polynomial in beta

Poly((2*H**2*alphabar*mubar - H**2*alphabar + 4*H**2*mubar**2 - 4*H**2*mubar + H**2 - 6*H*alphabar*mubar + 3*H*alphabar - 8*H*mubar**2 + 8*H*mubar - 2*H + 4*alphabar*mubar - 2*alphabar + 4*mubar**2 - 4*mubar + 1)*beta**2 + (-4*H**2*mubar*thetabar + 2*H**2*thetabar + 4*H*mubar*thetabar - 2*H*thetabar)*beta + H**2*thetabar**2, beta, domain='ZZ[H,alphabar,mubar,thetabar]')

Coefficients of K30 in beta

K30-coef of beta^2 = K302 =  (H - 1)*(2*mubar - 1)*(H*alphabar + 2*H*mubar - H - 2*alphabar - 2*mubar + 1)
K30-coef of beta^1 = K301 =  -2*H*thetabar*(H - 1)*(2*mubar - 1)
K30-coef of beta^0 = K300 =  H**2*thetabar**2

Clearly K300 > 0
Since (2*mubar-1)< 0 , it follows that K301 > 0
 
Also: if we denote  R =  (H*alphabar + 2*H*mubar - H - 2*alphabar - 2*mubar + 1)
then by adding H*alphabar, we get
(H - 1)*(2*alphabar + 2*mubar - 1)
R < R+ H*alphabar = (2H-1) =  (H - 1)*(2*alphabar + 2*mubar - 1)
Since 2*alphabar+2*mubar-1 < 0, it follows that the RHS < 0  and henc

##5.1.1.  $K(\gamma=1) = K_3+K_2+K_1+K_0 > 0$

It turns out that 

$$ K_3+K_2+K_1+K_0 = \beta H^2 (1-\overline{\mu})(\beta\overline{\mu} - \beta + \overline{\mu} + 1)^2  $$

which is positive since $\overline{\mu} < 1$.




In [13]:
# Show that K3+K2+K1+K0 > 0

K_at_1 = K0+K1+K2+K3

K_at_1 = K_at_1.expand().factor()

print('K(1) = K0+K1+K2+K3 =', K_at_1)


K(1) = K0+K1+K2+K3 = -H**2*beta*(mubar - 1)*(beta*mubar - beta + mubar + 1)**2


##5.1.2. $K(\gamma)$ is concave for $\gamma>1$, hence only has  zero-crossing greater than 1

Finally, we show that the cubic polynomial $K(\gamma)$ has only one zero-crossing for which $\gamma>1$. 
This then proves that the threshold $\overline{\gamma}_K$ is unique. 

To this end we prove that the 2nd derivative $K''(1) < 0$  showing 
that the cubic polynomial is concave to the right of $\gamma=1$. 

We first show that: 

$$ K''(1) = -2\overline{\alpha} \beta W = -2\overline{\alpha} \beta \left\{ W_2 \beta^2 + W_1\beta +W_0\right\}   $$

It turns out that the coefficients $W_0$ and $W_2$ are easily shown 
to be positive: 

  1. $ W_2 =  (H - 2)(H - 1)(1-\overline{\mu})(1-2(\overline{\alpha} + \overline{\mu}))  > 0 $

  2. $W_0 =  H^2\overline{\theta}(1+\overline{\mu} ) > 0 $


The $W_1$-coefficient requires more work. It turns out that 

$$ W_1 = -H W_{11} = -H(W_{111} \overline{\theta} + W_{110})  $$ where 

  1. $W_{111} = -(3H - 4)(1 - \overline{\mu}) < 0  $
  2. $ W_{110} =  - 2(H - 1)(\overline{\mu} + 1)(1-2\overline{\mu})  < 0 $

Since $\overline\mu < 1/2$ the linear function in $\overline{\theta}$ has negative slope and 
negative intercept, it has negative value for $\overline{\theta}>0$.  As a consequence we conclude that also $W_1 > 0$.

From the fact that $W_0, W_1, W_2 > 0$ it follows that $K''(1) < 0$  which proves 
that the threshold $\overline{\gamma}_K$ exists and is unique. 

In [14]:
# Since K(gamma) =  K3*gamma^3 + K2*gamma^2 + K1*gamma + K0
# it follows: 
#    
#    K'(gamma) = 3*K3*gamma^2 + 2*K2*gamma + K1
#   K''(gamma) = 6*K3*gamma   + 2*K2
#
#  and K'(0) = K1 
#   K''(1) = 6*K3 + 2*K2

#dK_dgamma = diff(K,gamma)

dK_dgamma_at_0 = K1

print('K1 = ', K1.factor())
print('K2 = ', K2.factor())

print('')
d2K_at_1 = 6*K3 + 2*K2
d2K_at_1 = d2K_at_1.expand().simplify().factor()


print("Introducing W defined by:  K''(1) = -alphabar*beta*W, we find: ")
W = (d2K_at_1/(-2*alphabar*beta)).cancel()

print('\n     W = ',W)

print('\nExpress W as a polynomial in beta: W = W2*beta^2 + W1*beta + W0')
poly_W = poly_from_expr(W,beta)[0]


# ????  what is K20 doing here??
#poly_K20 = poly_from_expr(K20,beta)[0]
# print('')
# print('Coefficients of K20 in beta')

print('\nExpanding the coefficients for the beta-powers: ')
W2 = (poly_W.all_coeffs()[0]).factor()
W1 = (poly_W.all_coeffs()[1]).factor()
W0 = (poly_W.all_coeffs()[2]).factor()

print('   W-coef of beta^2 = W2 = ', W2)
print('   W-coef of beta^1 = W1 = ', W1.factor())
print('   W-coef of beta^0 = W0 = ', W0)

print('')



print('We only need to investigate W1 (as W0 and W2 are obvious)')
print('\nIntroduce W11 by denoting  W1 = -H*W11')

W11 = (W1/(-H)).cancel()

print('   W11 = ', W11.collect(thetabar))

print('\nWe have to show that W11 < 0')

print('\nExpand W11 as function of thetabar:   W11 = W111*thetabar + W110')
poly_W11 = poly_from_expr(W11,thetabar)[0]

#print(poly_W11)

print('   Coeff of thetabar^1 = W111 = ' , poly_W11.all_coeffs()[0].factor())
print('   Coeff of thetabar^0 = W110 = ' , poly_W11.all_coeffs()[1].factor())

print('\nSince mubar< 1/2 both slope W111<0 and intercept W110<0 are negative')
print('\nAs a consequence:')
print('   W11 < 0, w1 = -H*W11 > 0 , W > 0,  K"(1) = -alphabar*beta*W < 0') 
print('This means that the cubic polynomial is in its concave part -- meaning there is only one zero-crossing greater than 1')


K1 =  -beta*(2*H**2*alphabar**2*beta**2*mubar + H**2*alphabar**2*beta**2 + 6*H**2*alphabar*beta**2*mubar**2 - 2*H**2*alphabar*beta**2*mubar - H**2*alphabar*beta**2 + 4*H**2*alphabar*beta*mubar**2 - 6*H**2*alphabar*beta*mubar*thetabar + 4*H**2*alphabar*beta*mubar - 2*H**2*alphabar*mubar*thetabar + 3*H**2*alphabar*thetabar**2 - 2*H**2*alphabar*thetabar + H**2*beta*mubar**2*thetabar - 2*H**2*beta*mubar*thetabar + H**2*beta*thetabar + H**2*mubar**2*thetabar - H**2*thetabar - 6*H*alphabar**2*beta**2*mubar - 3*H*alphabar**2*beta**2 - 10*H*alphabar*beta**2*mubar**2 + 2*H*alphabar*beta**2*mubar + 2*H*alphabar*beta**2 - 4*H*alphabar*beta*mubar**2 + 4*H*alphabar*beta*mubar*thetabar - 4*H*alphabar*beta*mubar + 2*H*alphabar*beta*thetabar + 4*alphabar**2*beta**2*mubar + 2*alphabar**2*beta**2 + 4*alphabar*beta**2*mubar**2 - alphabar*beta**2)
K2 =  alphabar*beta*(4*H**2*alphabar*beta**2*mubar - H**2*alphabar*beta**2 + 10*H**2*beta**2*mubar**2 - 9*H**2*beta**2*mubar + 2*H**2*beta**2 + 4*H**2*beta*muba

# 6. Condition on polynomial  $L(\gamma)$ 

**(KPD. section 4.2 & 4.3.2  and Appendix G)**

When risk-aversion is too severe, even probabilistic sniping won't pay.  The value for which this happens occurs when

(KPD.eq.42)
 $$\frac{ dN}{dp}|_{(p=0)} = 0$$
 Expanding  

$$\left. \frac{du^*(p)}{dp} \right|_{p=0} = \frac{N'(0)Q(0)-N(0)Q'(0)}{Q^2(0)}$$

 
and using that  $N(0)=0$ while   $Q(0)=D-C>0$, we 
see that eq. 
simplifies to: 
$$N'(0) = 0 $$


In [15]:
#  Express dN/dp(p=0) as a polynomial in q = \gamma-1: 

poly_dN_dp_0 = poly_from_expr(dN_dp_0,q)[0]
print("N'(0) = dN/dp(p=0) is a polynomial of degree  ", degree(poly_dN_dp_0,gen=q),  " in q = gamma-1")

print('')
print('Polynomial coefficients (in q):')
L2 = poly_dN_dp_0.all_coeffs()[0].factor()
L1 = poly_dN_dp_0.all_coeffs()[1].factor()
L0 = poly_dN_dp_0.all_coeffs()[2].factor()

print('Coef of q^2 = L2 = ', L2)
print('Coef of q^1 = L1 = ', L1)
print('Coef of q^0 = L0 = ', L0)



N'(0) = dN/dp(p=0) is a polynomial of degree   2  in q = gamma-1

Polynomial coefficients (in q):
Coef of q^2 = L2 =  -alphabar*beta*thetabar/2
Coef of q^1 = L1 =  0
Coef of q^0 = L0 =  -beta*(mubar - 1)*(beta*mubar - beta + mubar + 1)/2
